In [3]:
import requests, zipfile, io, re, pandas

api = 'https://api.earthref.org/v1/MagIC/{}'

### Given a MagIC contribution ID, download the MagIC contribution text file if it is public

In [9]:
# Try to download the contribution text for an invalid ID
response = requests.get(api.format('data'), headers={'Accept': 'application/json'}, params={'id': 'a'})
print(response.request.method, response.request.url)
print('Error:', response.json()['errors'][0]['message'], '\n')

# Try to download the contribution text for what could be valid, but are nonexistant, IDs
response = requests.get(api.format('data'), headers={'Accept': 'application/json'}, params={'id': [1000, 1001]})
print(response.request.method, response.request.url)
if (response.status_code == 204):
    print('Public contribution IDs 1000 and 1001 are not found in MagIC', '\n')
else:
    print('Error:', response.json()['errors'][0]['message'], '\n')

# Download the contribution text for a valid ID and print the first 200 characters
contribution_id = 16761
response = requests.get(api.format('data'), headers={'Accept': 'text/plain'}, params={'id': contribution_id})
print(response.request.method, response.request.url)
if (response.status_code == 200):
    contribution_text = response.text
    print(contribution_text[0:200], '\n')
elif (response.status_code == 204):
    print('Public contribution ID = \'{}\' not found in MagIC'.format(contribution_id), '\n')
else:
    print('Error:', response.json()['errors'][0]['message'], '\n')

# Get the ID for the latest public MagIC contribution
latest_contribution_id_response = requests.get(api.format('search/contributions?n_max_rows=1'))
print(latest_contribution_id_response.request.method, latest_contribution_id_response.request.url)
if (latest_contribution_id_response.status_code == 200):
    latest_contribution_id = latest_contribution_id_response.json()['results'][0]['id']
elif (latest_contribution_id_response.status_code == 204):
    print('Latest contribution ID not retrieved in MagIC', '\n')
else:
    print('Error:', latest_contribution_id_response.json()['errors'][0]['message'], '\n')

# Get the contribution text for the ID and print the first 200 characters
latest_contribution_response = requests.get(api.format('download'), headers={'Accept': 'application/zip'}, params={'id': latest_contribution_id})
print(latest_contribution_response.request.method, latest_contribution_response.request.url)
if (latest_contribution_response.status_code == 200):
    latest_contribution_zip = zipfile.ZipFile(io.BytesIO(latest_contribution_response.content))
    latest_contribution_text = io.TextIOWrapper(latest_contribution_zip.open('{}/magic_contribution_{}.txt'.format(latest_contribution_id, latest_contribution_id))).read()
    print(latest_contribution_text[0:200], '\n')
elif (latest_contribution_response.status_code == 204):
    print('Public contribution ID = \'{}\' not found in MagIC'.format(latest_contribution_id), '\n')
else:
    print('Error:', latest_contribution_response.json()['err'][0]['message'], '\n')

GET http://localhost:3200/v1/MagIC/data?id=a
Error: must be integer 

GET http://localhost:3200/v1/MagIC/data?id=1000&id=1001
Error: must be integer 

GET http://localhost:3200/v1/MagIC/data?id=16761
tab delimited	contribution
id	version	timestamp	contributor	data_model_version	reference
16761	2	2020-02-23T23:03:45.034Z	@ltauxe	3.0	10.1016/J.EPSL.2014.12.034
>>>>>>>>>>
tab delimited	locations
loca 

GET http://localhost:3200/v1/MagIC/search/contributions?n_max_rows=1
GET http://localhost:3200/v1/MagIC/download?id=19283
tab delimited	contribution
id	version	timestamp	contributor	data_model_version	reference	lab_names
19283	1	2017-10-04T10:40:37.613Z	@najmagic	3.0	10.1016/J.PRECAMRES.2013.06.006	Laboratory of the Main 



### Given a DOI, download the MagIC contribution text file for the latest public version(s)

In [10]:
# Download and extract the latest contribution file to a magic_contribution.txt file
reference_doi = '10.1016/J.PEPI.2017.05.009'
response = requests.get(api.format('download'), headers={'Accept': 'application/zip'}, params={'doi': reference_doi, 'n_max_contributions': 1})
print(response.request.method, response.request.url)
if (response.status_code == 200):
    contribution_zip = zipfile.ZipFile(io.BytesIO(response.content))
    for filename in contribution_zip.namelist():
        if (re.match(r'^\d+\/magic_contribution_\d+\.txt', filename)):
            contribution_text = io.TextIOWrapper(contribution_zip.open(filename)).read()
            with open('downloads/magic_contribution.txt', 'wt') as fh:
                fh.write(contribution_text)
            print(filename, 'extracted to downloads/magic_contribution.txt', '\n')
elif (response.status_code == 204):
    print('Public contribution with a reference DOI = \'{}\' not found in MagIC'.format(reference_doi), '\n')
else:
    print('Error:', response.json()['errors'][0]['message'], '\n')

# Get the contribution text for each matching contribution and print the first 200 characters
reference_doi = '10.1016/J.EPSL.2014.12.034'
response = requests.get(api.format('download'), headers={'Accept': 'application/zip'}, params={'doi': reference_doi, 'n_max_contributions': 2})
print(response.request.method, response.request.url)
if (response.status_code == 200):
    contribution_zip = zipfile.ZipFile(io.BytesIO(response.content))
    for filename in contribution_zip.namelist():
        if (re.match(r'^\d+\/magic_contribution_\d+\.txt', filename)):
            contribution_text = io.TextIOWrapper(contribution_zip.open(filename)).read()
            print(contribution_text[0:200], '\n')
elif (response.status_code == 204):
    print('Public contributions with a reference DOI = \'{}\' not found in MagIC'.format(reference_doi), '\n')
else:
    print('Error:', response.json()['errors'][0]['message'], '\n')

GET http://localhost:3200/v1/MagIC/download?doi=10.1016%2FJ.PEPI.2017.05.009&n_max_contributions=1
Error: Failed to retrieve contributions [16814] for download. 

GET http://localhost:3200/v1/MagIC/download?doi=10.1016%2FJ.EPSL.2014.12.034&n_max_contributions=2
Error: Failed to retrieve contributions [16761, 16760] for download. 



### Download a contribution file and validate it

In [16]:
contribution_id = 16901

response = requests.get(api.format('data'), params={'id': contribution_id})
print(response.request.method, response.request.url)
if (response.status_code == 200):
    contribution_file = 'downloads/magic_contribution_{}.txt'.format(contribution_id)
    open(contribution_file, 'w').write(response.text)
    print('Retrieved contribution data with ID', contribution_id, '\n')
    
    with open(contribution_file, 'rb') as f:
        validation_response = requests.post(api.format('validate'),
            headers={'Content-Type': 'text/plain'}, 
            data=f
        )

    print(validation_response.request.method, validation_response.request.url)
    if (validation_response.status_code == 200):
        validation_results = validation_response.json()['validation']
        print('Validated contribution with ID', contribution_id, ':\n', validation_results)
    else:
        print('Error Validating a Private Contribution:', validation_response.json()['errors'][0]['message'], '\n')
else:
    print('Retrieve Public Contribution Error:', response.json()['errors'][0]['message'], '\n')

GET http://localhost:3200/v1/MagIC/data?id=16901
Retrieved contribution data with ID 16901 

POST http://localhost:3200/v1/MagIC/validate
Validated contribution with ID 16901 :
 {'errors': [{'table': 'contribution', 'column': 'lab_names', 'message': 'The contribution table is missing required column "lab_names".', 'rows': [1]}], 'warnings': []}


### Get the 50 latest public sites in MagIC that mention basalt or sandstone

In [5]:
# Get the matching sites rows into a Pandas dataframe
response = requests.get(api.format('search/sites'), params={'query': 'basalt OR sandstone', 'n_max_rows': 50})
print(response.request.method, response.request.url)
if (response.status_code == 200):
    sites = response.json()['results']
elif (response.status_code == 204):
    print('No sites in MagIC match the search parameters.', '\n')
else:
    print('Error:', response.json()['errors'][0]['message'], '\n')

sites_df = pandas.DataFrame(sites)
sites_df.head(50)

GET http://localhost:3200/v1/MagIC/search/sites?query=basalt+OR+sandstone&n_max_rows=50


,dir_n_samples,criteria,description,lon,dir_k,geologic_types,method_codes,geologic_classes,int_abs,citations,...,vgp_n_samples,vgp_dp,vadm_n_samples,vdm_n_samples,vadm_sigma,vdm,analysts,int_n_specimens,dir_n_total_samples,int_n_total_specimens
0,8,DE-SPEC,Site direction included in results table,351.3069,401,Lava Flow,DA-DIR-GEO:DE-BFL:DE-FM:LP-DC5:LP-DIR-AF:LP-DI...,Extrusive:Igneous,,This study,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8,DE-SITE:DE-SPEC,"""VGP Site: jm009, """"Site VGP, coord system = g...",351.3069,401,Lava Flow,DA-DIR-GEO:DE-BFL:DE-FM:LP-DC5:LP-DIR-AF:LP-DI...,Extrusive:Igneous,,This study,...,8,5.2,,,,,NaN,NaN,NaN,NaN
2,6,DE-SPEC,Site direction included in results table,351.7157,146,Lava Flow,DA-DIR-GEO:DE-BFL:DE-BFP:DE-FM-LP:LP-DC5:LP-DI...,Extrusive:Igneous,,This study,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,DE-SITE:DE-SPEC,"""VGP Site: jm020, """"Site VGP, coord system = g...",351.7157,146,Lava Flow,DA-DIR-GEO:DE-BFL:DE-BFP:DE-FM-LP:LP-DC5:LP-DI...,Extrusive:Igneous,,This study,...,6,10.5,,,,,NaN,NaN,NaN,NaN
4,,IE-SITE,"V[A]DM: Site jm020, V[A]DM of site",351.7157,,Lava Flow,IE-TT:LP-PI-ALT-PTRM:LP-PI-BT-IZZI:LP-PI-BT-MD...,Extrusive:Igneous,7.55e-05,This study,...,NaN,NaN,2,2,7.71e+21,1.02e+23,NaN,NaN,NaN,NaN
5,5,DE-SPEC,Site direction included in results table,351.226,67,Lava Flow,DA-DIR-GEO:DE-BFL:DE-FM:LP-DC5:LP-DIR-AF:LP-DI...,Extrusive:Igneous,,This study,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,5,DE-SITE:DE-SPEC,"""VGP Site: jm023, """"Site VGP, coord system = g...",351.226,67,Lava Flow,DA-DIR-GEO:DE-BFL:DE-FM:LP-DC5:LP-DIR-AF:LP-DI...,Extrusive:Igneous,,This study,...,5,17.3,,,,,NaN,NaN,NaN,NaN
7,7,DE-SPEC,Site direction included in results table,351.5608,1209,Lava Flow,DA-DIR-GEO:DE-BFL:DE-BFP:DE-FM-LP:LP-DC5:LP-DI...,Extrusive:Igneous,,This study,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,7,DE-SITE:DE-SPEC,"""VGP Site: jm025, """"Site VGP, coord system = g...",351.5608,1209,Lava Flow,DA-DIR-GEO:DE-BFL:DE-BFP:DE-FM-LP:LP-DC5:LP-DI...,Extrusive:Igneous,,This study,...,7,3.3,,,,,NaN,NaN,NaN,NaN
9,6,DE-SPEC,Site direction included in results table,351.5653,795,Lava Flow,DA-DIR-GEO:DE-BFL:DE-BFP:DE-FM-LP:LP-DC5:LP-DI...,Extrusive:Igneous,,This study,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
